In [4]:
import numpy as np
import pandas as pd
import glob
import os

from scipy.io import loadmat
from scipy import stats

import seaborn as sns
import matplotlib.pyplot as plt

## Slit Sc

In [17]:
direc = r"./dataFolder/Submission 3/Final Submission/Data to be uploaded to Dryad/Figure2/"

In [18]:
file = pd.read_csv(direc + 'merged_TD', sep = "\t")
data = file.loc[:,('Treatment', 'Fly', 'wing_mean.Corleft', 'Average.Corleft', 'Average.Corright')]
data.columns = ['treatment', 'flyID', 'winglength', 'ClippedWingHz', 'IntactWingHz']

bins = [0.3, 0.5, 0.6, 0.7 ,0.9, 1]
data['binned'] = pd.cut(data['winglength'], bins, 
                        labels=np.array([0.3, 0.5, 0.6, 0.7, 0.9]))
new_df = data.drop(['winglength'], axis=1)

In [19]:
data.head()

,treatment,flyID,winglength,ClippedWingHz,IntactWingHz,binned
0,Intact,Fly1,0.821102,93.154528,93.349334,0.7
1,Intact,Fly1,0.666827,101.806114,102.028900,0.6
2,Intact,Fly1,0.573657,108.858352,108.810557,0.5
3,Intact,Fly1,0.461742,93.192249,92.950641,0.3
4,Intact,Fly1,0.391743,84.506527,83.927994,0.3


### compute the largest difference between clipped and intact wings for each fly

In [5]:
new_df = pd.DataFrame(data = None, columns = data.columns)

ii = 0
for trt, df in data.groupby('treatment'):
    for flyID, flydf in df.groupby('flyID'):
        temp = flydf.loc[(flydf['IntactWingHz'] - flydf['ClippedWingHz']).abs().idxmax(), :]
        new_df = new_df.append(temp, ignore_index = True)

In [6]:
new_df

,treatment,flyID,winglength,ClippedWingHz,IntactWingHz,binned
0,Intact,Fly1,0.391743,84.506527,83.927994,0.3
1,Intact,Fly2,1.000000,109.348125,112.487110,0.9
2,Intact,Fly3,0.759073,119.158525,119.983680,0.7
3,Intact,Fly5,0.620712,117.361145,117.687945,0.6
4,Intact,Fly6,0.720811,141.340205,140.790750,0.7
5,Intact,Fly7,0.582596,121.059640,120.698435,0.5
6,Slit,Fly1,0.617825,150.000000,110.000000,0.6
7,Slit,Fly2,0.407028,156.900000,86.270000,0.3
8,Slit,Fly7,0.516695,134.600000,84.620000,0.5
9,Slit,Fly8,0.453098,145.800000,83.330000,0.3


In [7]:
for trt, df in new_df.groupby('treatment'):
    clipw = df.ClippedWingHz.values  
    intactw = df.IntactWingHz.values
    U, p = stats.wilcoxon(clipw, intactw, alternative="greater")
    print('%s thorax contralateral wings, p value is %0.3f and U value is %0.3f' %(trt, p, U))

Intact thorax contralateral wings, p value is 0.623 and U value is 9.000
Slit thorax contralateral wings, p value is 0.022 and U value is 15.000


C:\Users\Tanvi\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\morestats.py:2863: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


In [22]:
# stats for each winglength

for trt, df_trt in data.groupby('treatment'):
    for binned, df_binned in df_trt.groupby('binned'):
        clipw = df_binned.ClippedWingHz.values  
        intactw = df_binned.IntactWingHz.values
        U, p = stats.wilcoxon(clipw, intactw)
        print('%s and %s, p value is %0.3f and U value is %0.8f' %(trt, binned, p, U))

Intact and 0.3, p value is 0.028 and U value is 0.00000000
Intact and 0.5, p value is 0.345 and U value is 6.00000000
Intact and 0.6, p value is 0.715 and U value is 4.00000000
Intact and 0.7, p value is 0.249 and U value is 5.00000000
Intact and 0.9, p value is 0.080 and U value is 1.00000000
Slit and 0.3, p value is 0.109 and U value is 0.00000000
Slit and 0.5, p value is 0.068 and U value is 0.00000000
Slit and 0.6, p value is 0.068 and U value is 0.00000000
Slit and 0.7, p value is 0.317 and U value is 0.00000000
Slit and 0.9, p value is 0.593 and U value is 2.00000000


In [9]:
data[(data.treatment == 'Intact') & (data.binned == 0.3)]

,treatment,flyID,winglength,ClippedWingHz,IntactWingHz,binned
3,Intact,Fly1,0.461742,93.192249,92.950641,0.3
4,Intact,Fly1,0.391743,84.506527,83.927994,0.3
9,Intact,Fly2,0.399406,105.311196,104.269030,0.3
14,Intact,Fly3,0.469790,89.220142,89.115336,0.3
19,Intact,Fly5,0.439044,126.764700,126.715680,0.3
29,Intact,Fly7,0.480691,136.785680,136.726155,0.3


In [11]:
data[(data.treatment == 'Slit') & (data.binned == 0.5)]

,treatment,flyID,winglength,ClippedWingHz,IntactWingHz,binned
37,Slit,Fly2,0.540692,110.7,83.03,0.5
42,Slit,Fly7,0.516695,134.6,84.62,0.5
46,Slit,Fly8,0.510228,130.1,84.55,0.5
51,Slit,Fly9,0.505850,143.9,84.21,0.5


## loaded haltere

In [11]:
direc = r"./dataFolder/Submission 3/Final Submission/Data to be uploaded to Dryad/HaltereLoading_Figure5/"

In [12]:
data = pd.read_csv(direc + 'compiledDataForAggregratePlots.csv')

In [13]:
data.head()

,Unnamed: 0,flyID,wingHz,leftHalHz,rightHalHz,treatment
0,0,fly1,96.904762,97.142857,96.904762,intacthaltere
1,1,fly1,102.130326,102.393484,102.393484,loadedhaltere1
2,2,fly1,112.452700,107.893285,112.125903,loadedhaltere2
3,3,fly1,109.064327,49.968041,109.356725,loadedhaltere3
4,4,fly1,109.064327,109.356725,109.064327,removedload


In [14]:
subset = data[data.treatment == 'loadedhaltere3']
subset

,Unnamed: 0,flyID,wingHz,leftHalHz,rightHalHz,treatment
3,3,fly1,109.064327,49.968041,109.356725,loadedhaltere3
8,8,fly2,85.540184,52.579224,85.552701,loadedhaltere3
13,13,fly3,91.182006,45.245099,91.433747,loadedhaltere3
18,18,fly4,108.919111,62.792650,109.449604,loadedhaltere3
23,23,fly5,90.389140,36.092735,90.532656,loadedhaltere3
28,28,fly8,93.741765,46.934931,93.939394,loadedhaltere3


In [16]:
# right control haltere, left loaded
rightHal= data.rightHalHz.values
leftHal = data.leftHalHz.values
wing = data.wingHz.values

U, p = stats.wilcoxon(leftHal, rightHal, alternative='less')
print('left-right haltere, p value is %0.5f and U value is %0.5f' %(p, U))

U, p = stats.wilcoxon(leftHal, wing, alternative='less')
print('left haltere-wing, p value is %0.5f and U value is %0.5f' %(p, U))

left-right haltere, p value is 0.00348 and U value is 84.50000
left haltere-wing, p value is 0.01795 and U value is 130.50000


## EpiRidge Cut

In [12]:
direc= r"./dataFolder/Submission 3/Final Submission/Data to be uploaded to Dryad/EpiRidgeCut_Figure3/"

In [13]:
data = pd.read_csv(direc + 'compiledDataForAggregratePlots.csv')

In [14]:
data.head()

,Unnamed: 0,flyID,treatment,wingHz,leftHalHz,rightHalHz,binned
0,0,fly1,cutwing1_1,138.095238,138.095238,138.095238,"(0.7, 0.9]"
1,1,fly1,cutwing2_1,132.321429,132.797619,132.310924,"(0.7, 0.9]"
2,2,fly1,cutwing3_1,151.648352,151.904762,151.831502,"(0.5, 0.6]"
3,3,fly1,cutwing4_1,138.387470,130.766537,137.388943,"(0.4, 0.5]"
4,4,fly1,cutwing5_1,187.272727,124.550654,125.486992,"(0.3, 0.4]"


In [27]:
subset = data[(data.binned == '(0.6, 0.7]')]
subset

,Unnamed: 0,flyID,treatment,wingHz,leftHalHz,rightHalHz,binned
7,7,fly2,cutwing2_1,116.462418,80.912690,117.622549,"(0.6, 0.7]"
13,13,fly3,cutwing2_1,133.928571,133.988095,133.809524,"(0.6, 0.7]"
19,19,fly5,cutwing2_1,136.058710,71.784505,104.832561,"(0.6, 0.7]"
24,24,fly6,cutwing2_2,113.939628,74.407437,128.014706,"(0.6, 0.7]"
29,29,fly7,cutwing2_1,143.699634,122.206435,144.102564,"(0.6, 0.7]"


In [26]:
wing = subset.wingHz.values
left = subset.leftHalHz.values
right = subset.rightHalHz.values

U, p = stats.wilcoxon(wing, left, alternative='greater')
print('wing and ridge cut haltere, p value is %0.5f and U value %0.3f' %(p,U))

U, p = stats.wilcoxon(wing, right, alternative='greater')
print('wing and control haltere, p value is %0.5f and U value %0.3f' %(p, U))

wing and ridge cut haltere, p value is 0.03981 and U value 14.000
wing and control haltere, p value is 0.65708 and U value 6.000


In [16]:
# do wilcoxon for each winglength

for binned, df_binned in data.groupby('binned'):
    print(binned)
    wing = df_binned.wingHz.values
    left = df_binned.leftHalHz.values
    right = df_binned.rightHalHz.values

    U, p = stats.wilcoxon(wing, left, alternative='greater')
    print('wing and ridge cut haltere, p value is %0.5f and U value %0.3f' %(p,U))

    U, p = stats.wilcoxon(wing, right, alternative='greater')
    print('wing and control haltere, p value is %0.5f and U value %0.3f' %(p, U))

(0.3, 0.4]
wing and ridge cut haltere, p value is 0.02156 and U value 15.000
wing and control haltere, p value is 0.02156 and U value 15.000
(0.4, 0.5]
wing and ridge cut haltere, p value is 0.02156 and U value 15.000
wing and control haltere, p value is 0.02156 and U value 15.000
(0.5, 0.6]
wing and ridge cut haltere, p value is 0.03981 and U value 14.000
wing and control haltere, p value is 0.25009 and U value 10.000
(0.6, 0.7]
wing and ridge cut haltere, p value is 0.03981 and U value 14.000
wing and control haltere, p value is 0.65708 and U value 6.000
(0.7, 0.9]
wing and ridge cut haltere, p value is 0.45826 and U value 11.000
wing and control haltere, p value is 0.54174 and U value 10.000
(0.9, 1.0]
wing and ridge cut haltere, p value is 0.96263 and U value 2.000
wing and control haltere, p value is 0.55363 and U value 7.000
